<a href="https://colab.research.google.com/github/Ciph3r007/ChatBot/blob/main/ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chatbot

- [1:   Dataset](#1)
- [2:   Preprocessing](#2)
    - [2.1:   Creating input pipeline](#2.1)
- [3:   Model Training](#4)
- [4:   Testing](#5)


<a name="1"></a>
# 1. The MultiWoz dataset

Installation and importing

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/My\ Drive/colab_data/chatbot/
!ls

In [3]:
!pip install -q trax

     |████████████████████████████████| 522kB 8.3MB/s 
     |████████████████████████████████| 3.4MB 16.4MB/s 
     |████████████████████████████████| 235kB 41.0MB/s 
     |████████████████████████████████| 61kB 8.4MB/s 
     |████████████████████████████████| 2.1MB 57.7MB/s 
     |████████████████████████████████| 1.2MB 54.6MB/s 
     |████████████████████████████████| 368kB 38.5MB/s 
     |████████████████████████████████| 3.9MB 43.3MB/s 
     |████████████████████████████████| 901kB 50.0MB/s 
     |████████████████████████████████| 3.3MB 52.8MB/s 


In [4]:
import json
import random
import numpy as np
from termcolor import colored

import trax   
from trax import layers as tl
from trax.supervised import training
!pip list | grep trax

trax                          1.3.7                


Dataset INFO

In [24]:
with open('data/README') as file:
    print(file.read())

#####################################################
#####################################################
#  Copyright Cambridge Dialogue Systems Group, 2018 #
#####################################################
#####################################################

Dataset contains the following files:
1. data.json: the woz dialogue dataset, which contains the conversation  users and wizards, as well as a set of coarse labels for each user turn. This file contains both system and user dialogue acts annotated at the turn level. Files with multi-domain dialogues have "MUL" in their names. Single domain dialogues have either "SNG" or "WOZ" in their names.
2. restaurant_db.json: the Cambridge restaurant database file, containing restaurants in the Cambridge UK area and a set of attributes.
3. attraction_db.json: the Cambridge attraction database file, contining attractions in the Cambridge UK area and a set of attributes.
4. hotel_db.json: the Cambridge hotel database file, containing

Declaring some CONSTANTS to be used later

In [6]:
DATA_FILE = 'data.json'
DATA_DIR = './data'
DIALOGUE_DB = {}

VOCAB_FILE = 'en_32k.subword'
VOCAB_DIR = 'data/vocabs'

Loading the MultiWoz dataset from json

In [7]:
def load_json(directory, file):
    with open(f'{directory}/{file}') as file: 
        db = json.load(file)
    return db
    
DIALOGUE_DB = load_json(DATA_DIR, DATA_FILE)

In [8]:
print(f'The number of dialogues is: {len(DIALOGUE_DB)}')

The number of dialogues is: 10438


The dialogues are composed of multiple files and the filenames are used as keys in the dictionary. Those with multi-domain dialogues have "MUL" in their filenames while single domain dialogues have either "SNG" or "WOZ".

In [9]:
print(list(DIALOGUE_DB.keys())[0:7]) 

['SNG01856.json', 'SNG0129.json', 'PMUL1635.json', 'MUL2168.json', 'SNG0073.json', 'SNG01445.json', 'MUL2105.json']


In [10]:
# get keys of the fifth file in the list above
print(DIALOGUE_DB['SNG0073.json'].keys())

dict_keys(['goal', 'log'])


Here `goal` points to a dictionary containing several key objectives of the conversation. `log` (a list) on the other hand contains the dialog in each of its item's `text` key.

In [11]:
DIALOGUE_DB['SNG0073.json']['goal']

{'attraction': {},
 'hospital': {},
 'hotel': {},
 'message': ["You want to book a <span class='emphasis'>taxi</span>. The taxi should go to <span class='emphasis'>pizza hut fen ditton</span> and should depart from <span class='emphasis'>saint john's college</span>",
  "The taxi should <span class='emphasis'>leave after 17:15</span>",
  "Make sure you get <span class='emphasis'>car type</span> and <span class='emphasis'>contact number</span>"],
 'police': {},
 'restaurant': {},
 'taxi': {'fail_info': {},
  'info': {'departure': "saint john's college",
   'destination': 'pizza hut fen ditton',
   'leaveAt': '17:15'},
  'reqt': ['car type', 'phone']},
 'train': {}}

In [12]:
DIALOGUE_DB['SNG0073.json']['log'][0]

{'metadata': {},
 'text': "I would like a taxi from Saint John's college to Pizza Hut Fen Ditton."}

The conversion goes between two persons back and forth

In [13]:
print(' Person 1: ', DIALOGUE_DB['SNG0073.json']['log'][0]['text'])
print(' Person 2: ',DIALOGUE_DB['SNG0073.json']['log'][1]['text'])

 Person 1:  I would like a taxi from Saint John's college to Pizza Hut Fen Ditton.
 Person 2:  What time do you want to leave and what time do you want to arrive by?


In [14]:
def get_conversation(file, data_db):
    result = ''
    len_msg_log = len(data_db[file]['log'])
    delimiter_1 = ' Person 1: '
    delimiter_2 = ' Person 2: '
    
    logs = data_db[file]['log']
    
    for i in range(len_msg_log):
        cur_log = logs[i]['text']
        
        if i % 2 == 0:
            result += delimiter_1
        else:
            result += delimiter_2
            
        result += cur_log

    return result

In [16]:
file = 'SNG01856.json'
conversation = get_conversation(file, DIALOGUE_DB)

print(conversation)

 Person 1: am looking for a place to to stay that has cheap price range it should be in a type of hotel Person 2: Okay, do you have a specific area you want to stay in? Person 1: no, i just need to make sure it's cheap. oh, and i need parking Person 2: I found 1 cheap hotel for you that includes parking. Do you like me to book it? Person 1: Yes, please. 6 people 3 nights starting on tuesday. Person 2: I am sorry but I wasn't able to book that for you for Tuesday. Is there another day you would like to stay or perhaps a shorter stay? Person 1: how about only 2 nights. Person 2: Booking was successful.
Reference number is : 7GAWK763. Anything else I can do for you? Person 1: No, that will be all. Good bye. Person 2: Thank you for using our services.


Prettifier function using termcolor

In [17]:
def print_conversation(conversation):
    
    delimiter_1 = 'Person 1: '
    delimiter_2 = 'Person 2: '
    
    split_list_d1 = conversation.split(delimiter_1)
    
    for sublist in split_list_d1[1:]:
        split_list_d2 = sublist.split(delimiter_2)
        print(colored(f'Person 1: {split_list_d2[0]}', 'red'))
        
        if len(split_list_d2) > 1:
            print(colored(f'Person 2: {split_list_d2[1]}', 'green'))

            
print_conversation(conversation)

Person 1: am looking for a place to to stay that has cheap price range it should be in a type of hotel 
Person 2: Okay, do you have a specific area you want to stay in? 
Person 1: no, i just need to make sure it's cheap. oh, and i need parking 
Person 2: I found 1 cheap hotel for you that includes parking. Do you like me to book it? 
Person 1: Yes, please. 6 people 3 nights starting on tuesday. 
Person 2: I am sorry but I wasn't able to book that for you for Tuesday. Is there another day you would like to stay or perhaps a shorter stay? 
Person 1: how about only 2 nights. 
Person 2: Booking was successful.
Reference number is : 7GAWK763. Anything else I can do for you? 
Person 1: No, that will be all. Good bye. 
Person 2: Thank you for using our services.


<a name="2"></a>
# 2. Preprocessing

In [25]:
all_files = DIALOGUE_DB.keys()
untokenized_data = []

for file in all_files:
    result = get_conversation(file, DIALOGUE_DB)
    untokenized_data.append(result)

print(untokenized_data[0])

 Person 1: am looking for a place to to stay that has cheap price range it should be in a type of hotel Person 2: Okay, do you have a specific area you want to stay in? Person 1: no, i just need to make sure it's cheap. oh, and i need parking Person 2: I found 1 cheap hotel for you that includes parking. Do you like me to book it? Person 1: Yes, please. 6 people 3 nights starting on tuesday. Person 2: I am sorry but I wasn't able to book that for you for Tuesday. Is there another day you would like to stay or perhaps a shorter stay? Person 1: how about only 2 nights. Person 2: Booking was successful.
Reference number is : 7GAWK763. Anything else I can do for you? Person 1: No, that will be all. Good bye. Person 2: Thank you for using our services.


Splitting the list to a train and eval dataset.

In [26]:
random.shuffle(untokenized_data)
cut_off = int(len(untokenized_data) * .05)
train_data, eval_data = untokenized_data[:-cut_off], untokenized_data[-cut_off:]

print(f'number of conversations in the data set: {len(untokenized_data)}')
print(f'number of conversations in train set: {len(train_data)}')
print(f'number of conversations in eval set: {len(eval_data)}')

number of conversations in the data set: 10438
number of conversations in train set: 9917
number of conversations in eval set: 521


<a name="2.1"></a>
## Creating input pipeline

In [27]:
def stream(data):
    while True:
        d = random.choice(data)
        yield (d, d)

Let's define our data pipeline for tokenizing and batching our data. We will also filter by maxlen and use bucketing for batch

In [28]:
data_pipeline = trax.data.Serial(
    trax.data.Shuffle(),
    trax.data.Tokenize(vocab_dir=VOCAB_DIR, vocab_file=VOCAB_FILE),
    trax.data.FilterByLength(2048),
    trax.data.BucketByLength(boundaries=[128, 256, 512, 1024],
                             batch_sizes=[16, 8, 4, 2, 1]),
    trax.data.AddLossWeights(id_to_mask=0)
)

train_stream = data_pipeline(stream(train_data))
eval_stream = data_pipeline(stream(eval_data))

Peek into the train stream.

In [29]:
# the stream generators will yield (input, target, mask_weights).
inp, _, _ = next(train_stream)
print("input shape: ", inp.shape)
print(trax.data.detokenize(inp[0], vocab_dir=VOCAB_DIR, vocab_file=VOCAB_FILE))

input shape:  (4, 512)
 Person 1: I am going to Cambridge and need a place to eat that serves lebanese food and is cheaply priced.  Person 2: Unfortunately there are no Lebanese restaurants in the cheap price range. Would you like another cuisine type or price range? Person 1: What kind of cheap restaurants are there in the center of town? Person 2: I'm sorry, there are none. Would you like to change your cuisine type or location? Person 1: how about one that serves indian food? Person 2: I found three cheap indian restaurants in the centre.  Would you like to book at Mahal of Cambridge? Person 1: That sounds great. I need a table for 6 at 16:30 on Saturday, please. And I will need the reference number once it's booked. Person 2: Booking was successful. The table will be reserved for 15 minutes. Your reference number is ZCRT70X4. Can I help with anything else? Person 1: I need to book a train on Sunday as well. Person 2: There are 404 entries for Sunday. Where would you like to depart 

<a name="3"></a>
# 3. Model Training

In [65]:
def ReformerLM(vocab_size=33000, n_layers=2, mode='train', attention_type=tl.SelfAttention):
    model = trax.models.reformer.ReformerLM(
        vocab_size=vocab_size,
        n_layers=n_layers,
        mode=mode,
        attention_type=attention_type
    )
    
    return model

In [61]:
temp_model = ReformerLM(mode='train')
print(str(temp_model))

del temp_model 

Serial[
  Serial[
    ShiftRight(1)
  ]
  Embedding_33000_512
  Dropout
  PositionalEncoding
  Dup_out2
  ReversibleSerial_in2_out2[
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        Dropout
        Serial[
          FastGelu
        ]
        Dense_512
        Dropout
      ]
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        Dropout
        Serial[
          FastGelu
        ]
        Dense_512
        Dropout
      ]
    ]
    ReversibleSwap_in2_out2
  ]
  Concatenate_in2
  LayerNorm
  Dropout
  Serial[
    Dense_33000
  ]
]


In [63]:
def training_loop(ReformerLM, train_gen, eval_gen, n_layers=2, output_dir = "./model/"):
    lr_schedule = trax.lr.warmup_and_rsqrt_decay(n_warmup_steps=1000, max_value=0.01)
    
    train_task = training.TrainTask(
        labeled_data=train_gen,
        loss_layer=tl.WeightedCategoryCrossEntropy(),
        optimizer=trax.optimizers.Adam(0.01),
        lr_schedule=lr_schedule,
        n_steps_per_checkpoint=10
    )
    
    eval_task = training.EvalTask(
        labeled_data=eval_gen,
        metrics=[tl.WeightedCategoryCrossEntropy(), tl.WeightedCategoryAccuracy()]
    )
    
    loop = training.Loop(model=ReformerLM(n_layers=n_layers),
                         tasks=[train_task],
                         eval_tasks=[eval_task],
                         output_dir=output_dir)
    
    return loop

Training the model

In [47]:
!rm -f model/model.pkl.gz
loop = training_loop(ReformerLM, train_stream, eval_stream, n_layers=6)
loop.run(100)


Step      1: Total number of trainable weights: 70673640
Step      1: Ran 1 train steps in 51.31 secs
Step      1: train WeightedCategoryCrossEntropy |  10.47020435
Step      1: eval  WeightedCategoryCrossEntropy |  10.41814709
Step      1: eval      WeightedCategoryAccuracy |  0.00000000

Step     10: Ran 9 train steps in 76.01 secs
Step     10: train WeightedCategoryCrossEntropy |  10.11940384
Step     10: eval  WeightedCategoryCrossEntropy |  9.58206272
Step     10: eval      WeightedCategoryAccuracy |  0.07237300

Step     20: Ran 10 train steps in 38.04 secs
Step     20: train WeightedCategoryCrossEntropy |  8.92206955
Step     20: eval  WeightedCategoryCrossEntropy |  7.84104586
Step     20: eval      WeightedCategoryAccuracy |  0.06186317

Step     30: Ran 10 train steps in 38.30 secs
Step     30: train WeightedCategoryCrossEntropy |  6.95686340
Step     30: eval  WeightedCategoryCrossEntropy |  6.06058884
Step     30: eval      WeightedCategoryAccuracy |  0.05801622

Step     

In [81]:
loop.run(1000)


Step    110: Ran 10 train steps in 39.01 secs
Step    110: train WeightedCategoryCrossEntropy |  5.58990049
Step    110: eval  WeightedCategoryCrossEntropy |  5.67875767
Step    110: eval      WeightedCategoryAccuracy |  0.06654344

Step    120: Ran 10 train steps in 39.58 secs
Step    120: train WeightedCategoryCrossEntropy |  5.62603760
Step    120: eval  WeightedCategoryCrossEntropy |  5.56408739
Step    120: eval      WeightedCategoryAccuracy |  0.06999353

Step    130: Ran 10 train steps in 39.96 secs
Step    130: train WeightedCategoryCrossEntropy |  5.61853743
Step    130: eval  WeightedCategoryCrossEntropy |  5.48290586
Step    130: eval      WeightedCategoryAccuracy |  0.06928105

Step    140: Ran 10 train steps in 39.63 secs
Step    140: train WeightedCategoryCrossEntropy |  5.58797169
Step    140: eval  WeightedCategoryCrossEntropy |  5.58695459
Step    140: eval      WeightedCategoryAccuracy |  0.07304461

Step    150: Ran 10 train steps in 40.01 secs
Step    150: train We

<a name="4"></a>
# 4. Testing

In [82]:
def attention(*args, **kwargs):
    # number of input positions to remember in a cache when doing fast inference. 
    kwargs['predict_mem_len'] = 120
    # number of input elements to drop once the fast inference input cache fills up.
    kwargs['predict_drop_len'] = 120
    # return the attention layer with the parameters defined above
    return tl.SelfAttention(*args, **kwargs)

# Getting the model with new attention for prediction
model = ReformerLM(
    vocab_size=33000,
    n_layers=6,
    mode='predict',
    attention_type=attention,
)

In [84]:
# TRAX needs the model to be initialized with this shape
shape11 = trax.shapes.ShapeDtype((1, 1), dtype=np.int32)
model.init(shape11)

# Loading weights from the trained model
model.weights = loop.eval_model.weights

# saving the starting state for each new dialogue prediction
STARTING_STATE = model.state

In [85]:
str(model) == str(loop.eval_model)

True

Utility functions

In [86]:
def tokenize(sentence, vocab_file, vocab_dir):
    return list(trax.data.tokenize(iter([sentence]), vocab_file=vocab_file, vocab_dir=vocab_dir))[0]

def detokenize(tokens, vocab_file, vocab_dir):
    return trax.data.detokenize(tokens, vocab_file=vocab_file, vocab_dir=vocab_dir)

In [87]:
def ReformerLM_output_gen(ReformerLM, start_sentence, vocab_file, vocab_dir, temperature):
    input_tokens = tokenize(start_sentence, vocab_file, vocab_dir)
    input_tokens_with_batch = input_tokens[None]
    
    # Using the autoregressive_sample_stream function from trax
    output_gen = trax.supervised.decoding.autoregressive_sample_stream( 
        model=ReformerLM,
        inputs=input_tokens_with_batch,
        temperature=temperature
    )
    
    return output_gen

In [91]:
def generate_dialogue(ReformerLM, model_state, start_sentence, vocab_file, vocab_dir, max_len, temperature):
    delimiter_1 = 'Person 1: ' 
    delimiter_2 = 'Person 2: '
    sentence = ''
    counter = 0
    
    result = [tokenize(': ', vocab_file=vocab_file, vocab_dir=vocab_dir)]
    
    ReformerLM.state = model_state
    
    output = ReformerLM_output_gen(ReformerLM, start_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, temperature=temperature)
    
    print(start_sentence.split(delimiter_2)[0].strip())
    
    for o in output:
        
        result.append(o)
        
        sentence = detokenize(np.concatenate(result, axis=0), vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR)
        
        if sentence.endswith(delimiter_1):
            sentence = sentence.split(delimiter_1)[0]
            print(f'{delimiter_2}{sentence}')
            sentence = ''
            result.clear()
        
        elif sentence.endswith(delimiter_2):
            sentence = sentence.split(delimiter_2)[0]
            print(f'{delimiter_1}{sentence}')
            sentence = ''
            result.clear()

        counter += 1
        
        if counter > max_len:
            break    



In [92]:
sample_sentence = ' Person 1: Are there theatres in town? Person 2: '
generate_dialogue(ReformerLM=model, model_state=STARTING_STATE, start_sentence=sample_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, max_len=120, temperature=0.2)

Person 1: Are there theatres in town?
Person 2: : I have a train 
Person 1: I am the I need to like to? 
Person 2: I': I need to help. 
Person 2: I am a centre. 
Person 1: I need a train 
Person 1: I you like the centre. 
Person 1: I need to like to like range. 


In [93]:
sample_sentence = ' Person 1: Is there a hospital nearby? Person 2: '
generate_dialogue(ReformerLM=model, model_state=STARTING_STATE, start_sentence=sample_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, max_len=120, temperature=0.2)

Person 1: Is there a hospital nearby?
Person 1: : I am you like to the 
Person 1: I have you need to like a 4 2: I have a and I am you you I need the 
Person 1: I need the 
Person 2: I have the train 
Person 2: I you the centre. 
Person 2: I have. 
Person 2: I need to. 
Person 1: The address, you like to the reference 1 you like to book the reference. 
Person 1: I help. 


In [94]:
sample_sentence = ' Person 1: Can you book a taxi? Person 2: '
generate_dialogue(ReformerLM=model, model_state=STARTING_STATE, start_sentence=sample_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, max_len=120, temperature=0.2)

Person 1: Can you book a taxi?
Person 2: : I need a a a a a to like. 
Person 1: I need. 
Person 1: I am have 1: I need the 
Person 1: I am the 
Person 1: 
Person 1: I am like to like to like to the centre. 
Person 2: I need. 
Person 2: I have the 
